### Importing GT

In [1]:
import json
import numpy as np
from scipy.spatial.transform import Rotation as R

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def create_obb_from_description(centroid, dimensions, rotations):
    # Extract centroid, dimensions, and rotations
    cx, cy, cz = centroid["x"], centroid["y"], centroid["z"]
    length, width, height = dimensions["length"], dimensions["width"], dimensions["height"]
    rx, ry, rz = rotations["x"], rotations["y"], rotations["z"]

    # Create the 8 corners of the box before rotation and translation
    dx = length / 2
    dy = width / 2
    dz = height / 2

    corners = np.array([
        [-dx, -dy, -dz],
        [ dx, -dy, -dz],
        [ dx,  dy, -dz],
        [-dx,  dy, -dz],
        [-dx, -dy,  dz],
        [ dx, -dy,  dz],
        [ dx,  dy,  dz],
        [-dx,  dy,  dz]
    ])

    # Apply rotations
    rotation = R.from_euler('xyz', [rx, ry, rz], degrees=True)
    rotated_corners = rotation.apply(corners)

    # Apply translation (centroid)
    translated_corners = rotated_corners + np.array([cx, cy, cz])

    return translated_corners

In [4]:
gt = json.load(open('../../labels/rtabmap_cloud.json'))
for obj in gt['objects']:
    obj['bbox'] = create_obb_from_description(obj['centroid'], obj['dimensions'], obj['rotations']).tolist()

In [5]:
predictions = json.load(open('predictions.json'))
predicted_objects = predictions['objects']

for obj in predicted_objects:
    obj['bbox'] = np.array(obj['OBB']).reshape(8, 3)
    obj['points'] = np.array(obj['points']).reshape(-1, 3)

## Visualize PC and BBoxes

In [6]:
import open3d as o3d
PC_PATH = '../../pointclouds/rtabmap_cloud.ply'

# Load the point cloud from a .ply file
pcd = o3d.io.read_point_cloud(PC_PATH)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
# Load GT bboxes
gt_boxes = []
for obj in gt['objects']:
    v = o3d.utility.Vector3dVector(obj['bbox'])
    bbox = o3d.geometry.OrientedBoundingBox.create_from_points(v)
    bbox.color = [0, 1, 0]
    gt_boxes.append(bbox)

# Load predicted bboxes
pred_boxes = []
for obj in predicted_objects:
    v = o3d.utility.Vector3dVector(obj['bbox'])
    try:
        bbox = o3d.geometry.OrientedBoundingBox.create_from_points(v)
        bbox.color = [1, 0, 0]
        pred_boxes.append(bbox)
    except:
        pass

In [8]:
# Visualize the point cloud
o3d.visualization.draw_geometries([pcd, *gt_boxes, *pred_boxes])

In [9]:
def compute_3d_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) of two 3D boxes.

    Parameters:
    - box1: (8, 3) numpy array of vertices for the first box.
    - box2: (8, 3) numpy array of vertices for the second box.

    Returns:
    - float: the IoU of the two boxes.
    """
    # Extract the min and max points
    min_point1 = np.min(box1, axis=0)
    max_point1 = np.max(box1, axis=0)
    min_point2 = np.min(box2, axis=0)
    max_point2 = np.max(box2, axis=0)

    # Calculate intersection bounds
    inter_min = np.maximum(min_point1, min_point2)
    inter_max = np.minimum(max_point1, max_point2)
    inter_dims = np.maximum(inter_max - inter_min, 0)

    # Intersection volume
    inter_volume = np.prod(inter_dims)

    # Volumes of the individual boxes
    volume1 = np.prod(max_point1 - min_point1)
    volume2 = np.prod(max_point2 - min_point2)

    # Union volume
    union_volume = volume1 + volume2 - inter_volume

    # Intersection over Union
    iou = inter_volume / union_volume if union_volume != 0 else 0

    return iou

In [10]:
# Associating GT and predicted objects
iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)))
for i, gt_box in enumerate(gt_boxes):
    for j, pred_box in enumerate(pred_boxes):
        iou_matrix[i, j] = compute_3d_iou(np.array(gt_box.get_box_points()), np.array(pred_box.get_box_points()))

# Find the best match for each GT object
gt_matches = np.argmax(iou_matrix, axis=1)
pred_matches = np.argmax(iou_matrix, axis=0)

gt_matches, pred_matches

(array([20, 18,  6,  9,  4,  3,  2, 19,  0, 20, 17, 16, 14, 12, 12, 22, 10,
        10]),
 array([ 0,  0,  6,  5,  4,  7,  2,  9, 13,  3, 16,  0, 13, 14, 12, 11, 11,
        10,  1,  7,  0,  2, 15,  0,  0,  0,  0,  0, 15,  0,  0,  0]))

In [11]:
# drawing only matching bboxes
matching_gt_boxes = [gt_boxes[i] for i in set(pred_matches)]
matching_pred_boxes = [pred_boxes[j] for j in set(gt_matches)]

o3d.visualization.draw_geometries([pcd, *matching_gt_boxes, *matching_pred_boxes])

In [12]:
# compute precision, recall, f1

def compute_precision_recall_f1(gt_boxes, pred_boxes, iou_threshold=0.5):
    """
    Compute the precision, recall, and F1 score of the predicted boxes.

    Parameters:
    - gt_boxes: list of GT boxes.
    - pred_boxes: list of predicted boxes.
    - iou_threshold: IoU threshold for a match.

    Returns:
    - float: precision.
    - float: recall.
    - float: F1 score.
    """
    # Compute the IoU matrix
    iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)))
    for i, gt_box in enumerate(gt_boxes):
        for j, pred_box in enumerate(pred_boxes):
            iou_matrix[i, j] = compute_3d_iou(np.array(gt_box.get_box_points()), np.array(pred_box.get_box_points()))

    # Find the best match for each GT object
    gt_matches = np.argmax(iou_matrix, axis=1)
    pred_matches = np.argmax(iou_matrix, axis=0)

    # Compute the number of true positives
    tp = np.sum(iou_matrix[np.arange(len(gt_boxes)), gt_matches] > iou_threshold)

    # Compute precision, recall, and F1
    precision = tp / len(pred_boxes) if len(pred_boxes) != 0 else 0
    recall = tp / len(gt_boxes) if len(gt_boxes) != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0

    return precision, recall, f1

In [13]:
precision, recall, f1 = compute_precision_recall_f1(gt_boxes, pred_boxes)
precision, recall, f1

(0.15625, 0.2777777777777778, 0.19999999999999998)

In [15]:
gt.keys()

dict_keys(['folder', 'filename', 'path', 'objects'])

In [17]:
save_me = {}
for key in gt.keys():
    if key != 'objects':
        save_me[key] = gt[key]
    else:
        save_me['objects'] = []
        for obj in gt['objects']:
            save_me['objects'].append({
                'name': obj['name'],
                'centroid': obj['centroid'],
                'dimensions': obj['dimensions'],
                'rotations': obj['rotations'],
            })

In [19]:
with open('gt.json', 'w') as f:
    json.dump(save_me, f)